In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from ipywidgets import interact, FloatSlider

In [ ]:
def f(t, a, b, c):
    return np.sqrt(a) * np.exp(-b * t) * np.sin(c * t) + 0.5 * np.cos(2 * t)


In [5]:
# Generate 10**3 pairs of (a, b, c) using LHS and transform as required
from pyDOE import *

lhs_samples=lhs(3, samples=100)
lhs_samples=np.array(lhs_samples)
lhs_samples[:][:,1]-=0.5
lhs_samples[:][:,2]*=5
lhs_samples[:][:,2]+=5

# time from 0 to 1 with 100 points
time = np.linspace(0, 1, 100)  

# Meshgrid of the (a,b,c) pairs
a_grid, b_grid, c_grid = np.meshgrid(lhs_samples[:][:,0], lhs_samples[:][:,1], lhs_samples[:][:,2])

# Evaluate the function for each pair of (a,b,c) for each time point
values = np.array([f(t, a_grid, b_grid, c_grid) for t in time])
values = values.reshape(len(time), -1)
# values.shape=(100, 100, 100, 100)

# Flatten the grid for use with griddata
points = np.array([a_grid.flatten(), b_grid.flatten(), c_grid.flatten()]).T
#a_flat.shape=(10000,)
#b_flat.shape=(10000,)
#c_flat.shape=(10000,)

# NEW CODE FOR GAUSSIAN PROCESS REGRESSION
# Define the kernel for the Gaussian Process
kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))

# Create the Gaussian Process Regressor
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=1)

# Fit the Gaussian Process for each time step
gp_models = [gp.fit(points, values[i]) for i in range(len(time))]


# Define new points where you want to interpolate
a_new = np.linspace(0, 1, 50)
b_new = np.linspace(-0.5, 0.5, 50)
c_new = np.linspace(5, 10, 50)
a_new, b_new, c_new = np.meshgrid(a_new, b_new, c_new)
points_new = np.array([a_new.flatten(), b_new.flatten(), c_new.flatten()]).T


MemoryError: Unable to allocate 7.28 TiB for an array with shape (1000000, 1000000) and data type float64

In [ ]:
# Predict the values for each time step
interpolated_values = np.array([gp_models[i].predict(points_new) for i in range(len(time))])

# Reshape the interpolated values to match the new grid
interpolated_values = interpolated_values.reshape(len(time), a_new.shape[0], a_new.shape[1], a_new.shape[2])

In [ ]:
# Function to plot the time series for specific a and b
def plot_time_series(a, b):
    # Find the closest indices in the new parameter grid
    a_idx = (np.abs(param1_new - a)).argmin()
    b_idx = (np.abs(param2_new - b)).argmin()
    
    # Extract the time series for the specific a and b
    time_series = interpolated_values[:, a_idx, b_idx]
    
    # Plot the time series
    plt.figure(figsize=(10, 6))
    plt.plot(time, time_series, label=f'a={a}, b={b}')
    plt.xlabel('Time')
    plt.ylabel('Function Value')
    plt.title('Time Series for Specific a and b')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create sliders for a and b
a_slider = FloatSlider(min=0, max=1, step=0.01, value=0.5, description='a')
b_slider = FloatSlider(min=0, max=1, step=0.01, value=0.5, description='b')

# Create an interactive plot
interact(plot_time_series, a=a_slider, b=b_slider)